In [1]:
import mlflow
import torch 
import torch.nn as nn 
import numpy as np 
import datetime
import ast
import pickle

In [2]:
from ExchnageEnv import MarketEnvironment

In [3]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"torch is running on : {device}")

torch is running on : cuda


In [4]:
mlflow_tracking_uri = 'file:/home/naradaw/dev/Charles_Schwab/code/RnD/v2/mlflow_experiments'
mlflow.set_tracking_uri(mlflow_tracking_uri)

experiment_name = "/portfolio-contructor-v3"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///home/naradaw/dev/Charles_Schwab/code/RnD/v2/mlflow_experiments/761903683111437854', creation_time=1730803455740, experiment_id='761903683111437854', last_update_time=1730803455740, lifecycle_stage='active', name='/portfolio-contructor-v3', tags={}>

In [4]:
data_path = "/home/naradaw/dev/Charles_Schwab/data/w_features/v1/2024_10_31/dataset_sqs_2024_10_31_11_18.pkl"

In [5]:
mlflow_tracking_uri = '/home/naradaw/dev/Charles_Schwab/code/RnD/v2/mlflow_experiments'
mlflow_registry_uri = '/home/naradaw/dev/Charles_Schwab/code/RnD/v2/mlflow_model_registry'

In [6]:
run_id = '7f84099719fe43a8896413c2bc70e049'

In [7]:
mlflow_client = mlflow.client.MlflowClient(
    tracking_uri=mlflow_tracking_uri,
    registry_uri= mlflow_registry_uri
)

In [8]:
run = mlflow_client.get_run(
    run_id=run_id
)

In [9]:
run_params = run.data.params
run_params

{'feature_set': "['close', 'volume_mfi', 'volume_fi', 'volume_nvi', 'trend_visual_ichimoku_a', 'trend_aroon_up', 'volatility_atr', 'trend_psar_down_indicator', 'volatility_bbl', 'volatility_dcl', 'trend_vortex_ind_neg', 'trend_cci', 'volume_sma_em', 'volatility_kch', 'volatility_kcl', 'trend_aroon_ind', 'volume_cmf', 'volatility_kcp', 'trend_ichimoku_base', 'momentum_stoch_rsi_d', 'momentum_ppo_signal', 'trend_ema_slow', 'volatility_bbhi', 'momentum_wr', 'trend_psar_up', 'volume_obv', 'others_dr', 'momentum_stoch', 'momentum_ppo', 'volatility_bbh', 'volatility_kcw', 'trend_adx_neg', 'volatility_dcm', 'trend_psar_up_indicator', 'momentum_pvo_signal', 'momentum_ppo_hist', 'trend_kst_diff', 'volume_vwap', 'volatility_bbm', 'momentum_uo', 'volatility_dcp', 'momentum_pvo_hist', 'trend_vortex_ind_diff', 'others_dlr', 'momentum_stoch_rsi', 'trend_macd_signal', 'trend_macd', 'momentum_kama', 'trend_dpo', 'others_cr', 'trend_visual_ichimoku_b', 'trend_kst', 'trend_adx', 'momentum_stoch_signal',

In [10]:
int(run_params['nheads'])

2

In [11]:
market_env = MarketEnvironment(
        device = device,
        data_path = data_path,
        holding_period = int(run_params['holding_period']),
        episode_duration = int(run_params['episode_duration']),
        train_test_split = float(run_params['train_test_split']),
        symbol_universe = ast.literal_eval(run_params['symbol_universe']),
        feature_set = ast.literal_eval(run_params['feature_set'])
        )

In [12]:
logged_model = 'runs:/7f84099719fe43a8896413c2bc70e049/portfolio_constructor_2024_11_06_11_26'

# Load model as a PyFuncModel.
loaded_model = mlflow.pytorch.load_model(logged_model)

MlflowException: Run '7f84099719fe43a8896413c2bc70e049' not found

In [ ]:
is_end = False
rewards = []
tran_costs = []

market_env.reset(mode = "test")
state = market_env.get_state()

while not is_end:
    _, allocations = loaded_model(state)
    state, reward, is_end, tran_cost = market_env.step(allocations)

    rewards.append(reward)
    tran_costs.append(tran_cost)

sharp_ratio = sharp_ratio_(rewards, tran_costs)
sharp_ratio

0.6429491909300373

In [ ]:
srem = loaded_model.SREM
caan = loaded_model.CAAN

In [ ]:
test_state = market_env.get_random_state()
test_state.shape

torch.Size([60, 20, 87])

In [ ]:
loaded_model(test_state)

(tensor([ 0,  2,  5,  9, 11, 12, 13, 15, 16, 19], device='cuda:0'),
 tensor([0.0583, 0.0000, 0.0832, 0.0000, 0.0000, 0.1546, 0.0000, 0.0000, 0.0000,
         0.0588, 0.0000, 0.0890, 0.1531, 0.1499, 0.0000, 0.0617, 0.0589, 0.0000,
         0.0000, 0.1324], device='cuda:0', grad_fn=<SoftmaxBackward0>))

# SREM test

In [ ]:
test_state[0].shape, test_state[0]

(torch.Size([20, 87]),
 tensor([[ 8.8540e+01,  4.2783e+01, -1.9243e+06,  ...,  1.2531e+01,
           0.0000e+00,  1.6014e-01],
         [ 8.6760e+01,  3.2799e+01, -1.0922e+06,  ...,  1.4025e+01,
           0.0000e+00,  1.9385e-01],
         [ 2.2145e+02,  2.7416e+01, -3.2654e+06,  ...,  1.5624e+01,
           1.0000e+00,  1.2687e+00],
         ...,
         [ 4.5926e+02,  5.8355e+01, -1.0878e+06,  ...,  1.0390e+01,
           0.0000e+00,  4.0466e+01],
         [ 5.3090e+01,  2.4255e+01, -1.0354e+07,  ...,  1.0195e+01,
           1.0000e+00,  1.8908e-05],
         [ 8.3890e+01,  1.8789e+01, -3.5972e+06,  ...,  3.0841e+01,
           0.0000e+00,  4.2571e-01]], device='cuda:0'))

In [ ]:
test_state_norm = loaded_model.layer_norm(test_state)
test_state_norm[0].shape, test_state_norm[0]

(torch.Size([20, 87]),
 tensor([[-0.1536, -0.1469, -0.4072,  ..., -0.1392, -0.1496, -0.1488],
         [-0.1064, -0.0997, -1.2051,  ..., -0.0916, -0.1024, -0.1018],
         [-0.1404, -0.1338, -1.2095,  ..., -0.1260, -0.1365, -0.1357],
         ...,
         [-0.1410, -0.1345, -0.4281,  ..., -0.1266, -0.1372, -0.1364],
         [-0.1263, -0.1196, -0.2196,  ..., -0.1117, -0.1223, -0.1216],
         [ 0.1658,  0.1729, -1.4826,  ...,  0.1832,  0.1698,  0.1698]],
        device='cuda:0', grad_fn=<SelectBackward0>))

In [ ]:
srem_out = srem(test_state)
srem_out.shape , srem_out

(torch.Size([20, 88]),
 tensor([[ 0.4909, -0.5810,  0.0867,  ...,  1.6540, -0.1321,  0.4858],
         [ 0.6827, -0.1134,  0.1896,  ...,  1.3003, -0.2117,  0.3038],
         [ 0.3185, -0.7783, -0.0292,  ...,  1.7337,  0.0156,  0.6156],
         ...,
         [ 0.6729, -0.2309,  0.1636,  ...,  1.3457, -0.2263,  0.3409],
         [ 0.7228, -0.0058,  0.1686,  ...,  1.0926, -0.2467,  0.2823],
         [-0.7512,  1.6830,  0.2535,  ..., -0.9058, -0.0551,  0.1124]],
        device='cuda:0', grad_fn=<MeanBackward1>))

In [ ]:
caan_out = caan(srem_out)
caan_out

tensor([ 0.2374, -0.2031,  0.5492,  0.0017, -0.3115,  1.1153, -0.1149,  0.0839,
        -0.4851,  0.1855, -0.1333,  0.5785,  1.1063,  1.0788, -0.3318,  0.0621,
         0.1934, -0.1594, -0.3160,  0.8598], device='cuda:0',
       grad_fn=<SqueezeBackward0>)